
# GAUSSIAN NAIVE BAYES 

## TAREA 2

Sonia Mendizábal 105720

26 de agosto de 2016

In [2]:
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pda
import numpy as np
import scipy.stats
%matplotlib inline

In [3]:
dat = pda.read_table("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data", 
                    sep=",", header=None)
df = pda.DataFrame(dat)
pspam = np.mean(dat[:][57])
print pspam

0.394044772875


In [118]:
X_train, X_test = train_test_split(df, train_size=0.75)
dftrain = pda.DataFrame(X_train)
dftest = pda.DataFrame(X_test)
dftrain.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
705,0.00,0.00,0.00,0.0,0.00,0.00,4.00,0.00,0.00,0.00,...,0.0,0.000,0.000,0.000,0.000,0.000,8.166,19,49,1
2006,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.75,...,0.0,0.800,0.133,0.000,0.000,0.133,4.472,33,161,0
218,0.17,0.26,1.07,0.0,0.35,0.62,0.53,0.17,0.62,0.80,...,0.0,0.066,0.000,0.212,0.185,0.013,6.815,583,1329,1
1078,0.00,1.56,0.00,0.0,0.00,0.00,1.56,0.00,0.00,1.56,...,0.0,0.183,0.000,0.000,0.183,0.183,11.714,140,164,1
2136,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,1.01,...,0.0,0.000,0.312,0.000,0.000,0.000,4.030,28,133,0


In [22]:
tabm0 = np.mean(dftrain[dftrain[57] == 0])[:57]
tabs0 = np.std(dftrain[dftrain[57] == 0])[:57]
tabm1 = np.mean(dftrain[dftrain[57] == 1])[:57]
tabs1 = np.std(dftrain[dftrain[57] == 1])[:57]
tabms = pda.DataFrame({'mean 0' : tabm0,
              'std 0' : tabs0, 
              'mean 1' : tabm1, 
              'std 1' :tabs1})

In [48]:
tabms.head()

,mean 0,mean 1,std 0,std 1
0,0.072944,0.146754,0.295664,0.308770
1,0.224655,0.163853,1.526013,0.360494
2,0.198001,0.397024,0.504149,0.476395
3,0.000855,0.199235,0.022897,2.521261
4,0.183511,0.522691,0.647309,0.726526


In [119]:
def CeroTransf(sdvec):
    sdveccor = []
    for x in sdvec: 
        if x == 0: 
            sdveccor.append(x + .0000001)
        else:
            sdveccor.append(x)
    return(sdveccor)

def NaiveBayesRow(vec):
    
    normvecnspam = scipy.stats.norm(tabms[:][tabms.columns[0]],
                 CeroTransf(tabms[:]['std 0'] )
                ).pdf(vec) 
    normvecspam = scipy.stats.norm(tabms[:][tabms.columns[1]],
                 CeroTransf(tabms[:]['std 1'] )
                ).pdf(vec) 
        
    transpam = np.log(pspam) + np.sum(np.log(normvecspam))
    trannspam = np.log(1-pspam) + np.sum(np.log(normvecnspam))
    transmaximo = np.maximum(transpam, trannspam)
    #if transpam == transmaximo:
    if transpam >= trannspam:
        res = 1
    else:
        res = 0
    return(res)

def NaiveBayes(tabla):
    resl = []
    for i in range(len(tabla.index)):
        vec = tabla[i-1:i][tabla.columns[:57]]
        resl.append(NaiveBayesRow(vec))
    return(resl)

In [120]:
prediccion = NaiveBayes(dftrain)

In [121]:
100*np.sum(prediccion == dftrain[dftrain.columns[57]])/len(prediccion)

50

In [122]:
confusion_matrix(dftrain[dftrain.columns[57]], prediccion)

array([[ 980, 1115],
       [ 596,  759]])

In [123]:
predicciontest = NaiveBayes(dftest)

In [124]:
100*np.sum(predicciontest == dftest[dftest.columns[57]])/len(predicciontest)

48

In [125]:
confusion_matrix(dftest[dftest.columns[57]], predicciontest)

array([[318, 375],
       [218, 240]])

### Prueba con Función de SKLearn

In [126]:
clf = GaussianNB()
clf.fit(dftrain[dftrain.columns[:57]], dftrain[dftrain.columns[57]])
prediccionsktrain = clf.predict(dftrain[dftrain.columns[:57]])
prediccionsktest = clf.predict(dftest[dftest.columns[:57]])

In [127]:
sum(prediccionsktrain == dftrain[dftrain.columns[57]])*100/len(prediccionsktrain)

82

In [128]:
confusion_matrix(dftrain[dftrain.columns[57]], prediccionsktrain)

array([[1533,  562],
       [  52, 1303]])

In [129]:
sum(prediccionsktest == dftest[dftest.columns[57]])*100/len(prediccionsktest)

83

In [130]:
 confusion_matrix(dftest[dftest.columns[57]], prediccionsktest)

array([[520, 173],
       [ 19, 439]])